In [17]:
import os, sys
import hashlib
import datetime
import time
import json

In [29]:
class Block:
    PROOF_OF_WORK_HASH_PREFIX = '0'
    
    def __init__(self, index, timestamp, data, proof, previous_hash):
        self.bm_index = index
        self.bm_timestamp = timestamp
        self.bm_data = data
        self.bm_proof = proof
        self.bm_previous_hash = previous_hash
        self.hash = self.compute_hash()
        
    def to_json(self):
        d = {k: getattr(self, k) for k in dir(self) if k.startswith('bm_')}
        return json.dumps(d, sort_keys=True)
    
    def compute_hash(self):        
        return hashlib.sha256(self.to_json().encode()).hexdigest()
    
    def mine_next_proof(self, timeout_secs=10):        
        start_ts = time.time()
        loop_count = 0
        print(f'Mining next proof starting @: {self.bm_proof}')

        while True:
            loop_count += 1
            if loop_count % 1000 == 0:
                if time.time() > (start_ts + timeout_secs):
                    raise Exception(f"POWTimeout: Cound not mine next proof in f{timeout_secs}")
            self.bm_proof += 1
            self.hash = self.compute_hash()
            if self.is_valid():
                print(f'Found proof {self.bm_proof}')
                return True

    def is_valid(self):
        return self.hash.startswith(self.PROOF_OF_WORK_HASH_PREFIX)
    
    def __str__(self):   
        return f"Block: {self.to_json()} Hash: {self.hash}"
    

class BlockChain:
    GENESIS_BLOCK_DATA = "Genesis Block" 
    GENESIS_PREVIOUS_HASH = '0' * 64
    GENESIS_PROOF = 0
    CHAIN_NAME = "TryCoinChain"
    __BlockChainStore = []
    

    def __init__(self):
        pass

    def add_block(self, data):
        index = len(self.__BlockChainStore)
        if index:
            proof = self.__BlockChainStore[-1].bm_proof
            previous_hash = self.__BlockChainStore[-1].hash
        else:
            proof = self.GENESIS_PROOF
            previous_hash = self.GENESIS_PREVIOUS_HASH
            
        new_block = Block(index=len(self.__BlockChainStore), timestamp=time.time(), data=data, 
                         proof=proof, previous_hash=previous_hash)
        new_block.mine_next_proof()
        self.__BlockChainStore.append(new_block)
        return new_block
        
    
    def initialize_ico_blocks(self, number_of_blocks=20):
        # initialize with genesis block
        print(f"Creating Genesis Block")
        self.add_block(self.GENESIS_BLOCK_DATA)
        
        print(f"Adding initial {number_of_blocks} blocks")
        for i in range(number_of_blocks):
            data = {'TXN': i}            
            self.add_block(data)
        
    def validate_chain(self):
        for i in range(len(self.__BlockChainStore)):
            block = self.__BlockChainStore[i]
            #check previous_hash is correct
            if i == 0: #GENESIS
                expected_previous_hash = self.GENESIS_PREVIOUS_HASH
            else:
                expected_previous_hash = self.__BlockChainStore[i-1].compute_hash()
            if expected_previous_hash != block.bm_previous_hash:
                printf("BLOCKCHAIN_VALIDATION_ERROR: at location {i} - previous hash match failure")
                return False
            if block.hash != block.compute_hash():
                printf("BLOCKCHAIN_VALIDATION_ERROR: at location {i} -  stored hash and computed hash do not match")
                return False
            if not block.is_valid():
                printf("BLOCKCHAIN_VALIDATION_ERROR: at location {i} -  POW Validation failed")
                return False
        return True
            
            
       

trycoin = BlockChain()
trycoin.initialize_ico_blocks()


Creating Genesis Block
Mining next proof starting @: 0
Found proof 6
Adding initial 20 blocks
Mining next proof starting @: 6
Found proof 18
Mining next proof starting @: 18
Found proof 36
Mining next proof starting @: 36
Found proof 40
Mining next proof starting @: 40
Found proof 43
Mining next proof starting @: 43
Found proof 63
Mining next proof starting @: 63
Found proof 65
Mining next proof starting @: 65
Found proof 100
Mining next proof starting @: 100
Found proof 108
Mining next proof starting @: 108
Found proof 111
Mining next proof starting @: 111
Found proof 165
Mining next proof starting @: 165
Found proof 179
Mining next proof starting @: 179
Found proof 201
Mining next proof starting @: 201
Found proof 227
Mining next proof starting @: 227
Found proof 266
Mining next proof starting @: 266
Found proof 276
Mining next proof starting @: 276
Found proof 282
Mining next proof starting @: 282
Found proof 285
Mining next proof starting @: 285
Found proof 312
Mining next proof st

In [30]:
trycoin.validate_chain()
    

True